## Podcast Audio

https://www.datacamp.com/tutorial/amazon-polly

In [14]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from IPython.display import Markdown
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent

In [15]:
model = init_chat_model(
    "gemini-2.0-flash", model_provider="google_genai")

# .with_structured_output(MyOutput)
response = model.invoke("Hello World!!")
print(response)

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--92b7bf8d-980a-44b1-aca7-ed43a46cfcd6-0' usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}}


In [16]:
# Student profile type definition
class StudentProfile(TypedDict):
    gender: str
    grade_level: str  # e.g., "class 6", "12th", "undergrad", "postgrad"
    language: str     # e.g., "hindi", "english", "marathi"


class UserPrompt(TypedDict):
    topic: str
    pdf_file: str
    audio_file: str

# Graph state


class State(TypedDict):
    student_profile: StudentProfile
    user_prompt: UserPrompt
    summary_notes: str
    podcast_script: str
    mindmap: str
    quiz: str
    recommended_resources: str
    study_plan: str
    combined_output: str

In [17]:
# Prompts Templates
class PersonalizedPromptsStudent:
    def __init__(self, topic, student_profile: dict, user_instructions: str = ""):
        self.topic = topic
        self.student_profile = student_profile
        self.user_instructions = user_instructions

    def recommendation_resources(self):
        """Create a personalized prompt based on student profile"""

        prompt_template = ChatPromptTemplate([
    ("system", """
         You are an expert academic female tutor. Your goal is to generate engaging, informative, and personalized educational summaries for students.

        **Strict Guidelines:**

        1.  **Output Format:** The entire response MUST be valid SSML (Speech Synthesis Markup Language).
            * Start with `<speak>`.
            * End with `</speak>`.
            * Use only the SSML tags listed below. Other tags are NOT allowed.

        2.  **Content Length & Duration:**
            * Generate content for a concise audio overview, aiming for a duration of approximately 7-10 minutes.
            * **Crucially, the raw character count of the final SSML string (including all tags, spaces, and content) MUST NOT exceed 3000 characters.** If your content draft exceeds this, you MUST self-edit and shorten it to fit, prioritizing key information.

        **Student Profile:**
        - Class Level: {grade_level}
        - Target Language: {language} (Provide ALL content, including SSML elements like text and prompts, in this language.)
        - Preferred Pronouns: {gender} (e.g., she/her, he/him, they/them – use to personalize direct address, if appropriate for the content style.)

        **Content Requirements:**
        * **Topic Summary:** Generate a detailed and well-structured summary of the user-provided topic.
        * **Academic Appropriateness:** Tailor the depth, complexity, and vocabulary precisely to the specified {grade_level}. Assume foundational knowledge typical for their level, but introduce new concepts clearly.
        * **Engaging Delivery Style:**
            * Write in a conversational, accessible, and enthusiastic tone.
            * Incorporate brief, relatable examples or analogies where helpful.
            * Suggest pauses, changes in speaking rate, or emphasis using SSML tags to enhance listening experience.
        * **Structure:** Follow a logical flow:
            * **Introduction:** Hook the listener, introduce the topic, and greet them.
            * **Main Content:** Break down the topic into digestible segments, explaining core concepts.
            * **Key Takeaways/Recap:** Briefly summarize the main points.
            * **Call to Action/Further Exploration:** Encourage continued learning.

        **Allowed SSML Tags for Amazon Polly Neural Voices (Strictly Enforced):**

        * `<break>`: For adding pauses. Use `time` attribute (e.g., `<break time="500ms"/>` or `<break time="0.5s"/>`).
        * `<lang>`: To specify another language for specific words or phrases.
        * `<mark>`: To place a custom tag in your text.
        * `<p>`: To add a pause between paragraphs.
        * `<phoneme>`: For using phonetic pronunciation.
        * `<prosody>`: For controlling volume, speaking rate, and pitch. **Do NOT use `amazon:max-duration` attribute.** Use attributes like `rate`, `pitch`, `volume`.
        * `<s>`: To add a pause between sentences.
        * `<speak>`: The root element (must be the outermost tag).
        * `<sub>`: To pronounce acronyms and abbreviations (e.g., `<sub alias="Application Programming Interface">API</sub>`).
        * `<w>`: To improve pronunciation by specifying parts of speech (e.g., `<w role="verb">read</w>`).
        * `<amazon:effect name="drc">`: For adding dynamic range compression.

        **Do NOT use any other SSML tags, including (but not limited to):**
        * `<emphasis>`
        * `<prosody amazon:max-duration>`
        * `<amazon:auto-breaths>`
        * `<amazon:effect phonation="soft">`
        * `<amazon:effect vocal-tract-length>`
        * `<amazon:effect name="whispered">`
        * `<amazon:domain name="news">`

        **Example SSML structure and character count consideration:**
        <speak version="1.0" xml:lang="en-US">
          <p>Hello! Welcome to our podcast.</p>
          <s>Today, we explore [Topic].</s>
          <p>This is a brief explanation.</p>
          <s>We'll keep it concise.</s>
          <break time="200ms"/>
          <p><prosody rate="fast">Key takeaway:</prosody> It's very simple.</p>
          <s>Thanks for listening!</s>
        </speak>
        (This example is ~250 chars. Your full content should be around 3000 chars.)
        """
    ),
    ("user", "Generate an audio summary about: {topic_summary}")
        ])
        
        prompt_template_v2 = ChatPromptTemplate([
            ("system", """
         You are an expert academic female tutor. Your goal is to generate engaging, informative, and personalized educational summaries for students.

            Student Profile:
            - Class Level: {grade_level}
            - Target Language: {language} (Provide content in this language.)
            - Preferred Pronouns: {gender} 

            Podcast Content Requirements:
            1.  Comprehensive Summary: Generate a detailed and well-structured summary of the user-provided topic. This summary should serve as the core script for a 7-10 minute audio podcast episode within a 3000 character limit.
            2.  Academic Appropriateness: Tailor the depth, complexity, and vocabulary of the content precisely to the specified {grade_level}. Assume the student has foundational knowledge typical for their level, but introduce new concepts clearly.
            3.  Engaging Delivery Style:
                - Write in a conversational, accessible, and enthusiastic tone.
                - Incorporate brief, relatable examples or analogies where helpful.
                - Include a brief, friendly introduction and conclusion suitable for a podcast.
            4.  Structure: Your summary should implicitly or explicitly follow a logical podcast flow:
                - Introduction: Hook the listener, introduce the topic.
                - Main Content: Break down the topic into digestible segments.
                - Key Takeaways/Recap: Briefly summarize the main points.
                - Call to Action/Further Exploration:** Encourage continued learning.
            5. Strictly use the 3000 characters limit, if the script goes beyond this, self edit it to adhere to the character limit.
                """),

            ("user", "Create a audio summary for the topic: {topic}.")
        ])

        return prompt_template_v2.invoke({
            "topic": self.topic,
            "grade_level": self.student_profile.get("grade_level", "general"),
            "language": self.student_profile.get("language", "English"),
            "gender": self.student_profile.get("gender", ""),
        })

In [18]:
class PodcastContent(BaseModel):
    topic: str = Field(description="topic of the podcast")
    # SSML: str = Field(description="SSML content for the podcast")
    script: str = Field(description="Script for the audio summary")
    

In [19]:
def node_recommendation_sources(state: State):
    """LLM call to generate recommendation materials"""

    prompt_instance = PersonalizedPromptsStudent(
        state['user_prompt']['topic'], state['student_profile'])

    prompt_message = prompt_instance.recommendation_resources()
        
    model = init_chat_model(
        "gemini-2.0-flash", model_provider="google_genai").with_structured_output(PodcastContent)

    response = model.invoke(prompt_message)
    
    print(response)

    return {"response": response}

In [20]:
input_state = {
    "student_profile": {
        "gender": "MALE",
        "grade_level": "12th",
        "language": "hindi"
    },
    "user_prompt": {
        "topic": "Chola kingdom"
    }
}

In [21]:
output = node_recommendation_sources(input_state)

topic='Chola kingdom' script='नमस्ते दोस्तों! आज हम बात करेंगे चोल साम्राज्य के बारे में, जो दक्षिण भारत का एक शक्तिशाली और महत्वपूर्ण साम्राज्य था। चोलों ने लगभग 9वीं शताब्दी से 13वीं शताब्दी तक शासन किया, और उनका प्रभाव कला, वास्तुकला, साहित्य और प्रशासन में दिखता है। उन्होंने तंजावुर और गंगईकोंड चोलपुरम जैसे भव्य मंदिरों का निर्माण किया। उनका नौसेना बल बहुत मजबूत था, जिससे उन्होंने समुद्री व्यापार को बढ़ावा दिया और श्रीलंका जैसे क्षेत्रों पर नियंत्रण स्थापित किया। चोल साम्राज्य ने स्थानीय स्वशासन को प्रोत्साहित किया, जिसमें ग्राम सभाओं का महत्वपूर्ण योगदान था। संक्षेप में, चोल साम्राज्य दक्षिण भारत के इतिहास का एक स्वर्णिम अध्याय है। अगले एपिसोड में फिर मिलेंगे!'


In [22]:
output["response"].model_dump()

{'topic': 'Chola kingdom',
 'script': 'नमस्ते दोस्तों! आज हम बात करेंगे चोल साम्राज्य के बारे में, जो दक्षिण भारत का एक शक्तिशाली और महत्वपूर्ण साम्राज्य था। चोलों ने लगभग 9वीं शताब्दी से 13वीं शताब्दी तक शासन किया, और उनका प्रभाव कला, वास्तुकला, साहित्य और प्रशासन में दिखता है। उन्होंने तंजावुर और गंगईकोंड चोलपुरम जैसे भव्य मंदिरों का निर्माण किया। उनका नौसेना बल बहुत मजबूत था, जिससे उन्होंने समुद्री व्यापार को बढ़ावा दिया और श्रीलंका जैसे क्षेत्रों पर नियंत्रण स्थापित किया। चोल साम्राज्य ने स्थानीय स्वशासन को प्रोत्साहित किया, जिसमें ग्राम सभाओं का महत्वपूर्ण योगदान था। संक्षेप में, चोल साम्राज्य दक्षिण भारत के इतिहास का एक स्वर्णिम अध्याय है। अगले एपिसोड में फिर मिलेंगे!'}

In [ ]:
dict_response = output["response"].model_dump()
SSML = dict_response['SSML']
SSML

In [ ]:
import re

def clean_podcast_content(content):
    clean_text = re.sub(r"[\n\r]", "", content)
    return clean_text

SSML = clean_podcast_content(SSML)

SSML

## Generate Podcast Audio using AWS

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/polly/client/synthesize_speech.html#

In [ ]:
import boto3

polly = boto3.client('polly')

In [ ]:
text_input = "<speak>नमस्ते! मेरा नाम काजल। मैं आपके द्वारा यहां टाइप किए गए किसी भी वाक्य को पढ़ सकती हूं।</speak>"

In [ ]:
text_input = '<speak><p>नमस्ते! आज हम AI में अनिश्चितता के बारे में बात करेंगे।</p><s>यह मॉड्यूल AI सिस्टम्स में अनिश्चितता को मापने और प्रबंधित करने के तरीकों पर केंद्रित है।</s><break time="0.5s"/><p>AI एजेंटों को अनिश्चितता से निपटने की आवश्यकता होती है क्योंकि वे हमेशा यह नहीं जानते कि वे किस स्थिति में हैं या कार्यों के बाद कहां पहुंचेंगे।</p><s>वे belief state का ट्रैक रखकर अनिश्चितता को संभालते हैं।</s><break time="0.3s"/><p>उदाहरण के लिए, एक ऑटोमेटेड टैक्सी को हवाई अड्डे पर समय पर यात्री को पहुंचाने का लक्ष्य है, लेकिन उसे रास्ते में कई अनिश्चितताओं का सामना करना पड़ता है।</p><break time="0.5s"/><p>अनिश्चित reasoning का एक उदाहरण dental diagnosis है, जिसमें हमेशा अनिश्चितता शामिल होती है।</p><s>अनिश्चितता से निपटने के लिए probability theory सबसे अच्छा उपकरण है।</s><break time="0.3s"/><p>Diagnosis के समय, वास्तविक दुनिया में कोई अनिश्चितता नहीं होती: रोगी को या तो cavity होती है या नहीं।</p><break time="0.5s"/><p>सफलता दरों की तुलना करना, reliability का महत्व, trade-offs, preferences और outcomes, utility theory, और Maximum Expected Utility (MEU) का सिद्धांत महत्वपूर्ण हैं।</p><break time="0.5s"/><p>Probability Assertions में Random variables, domain, probability distribution, joint probability distribution, और full joint probability distribution महत्वपूर्ण हैं।</p><break time="0.5s"/><p>Probability Axioms में Non-negativity axiom और normalization axiom शामिल हैं।</p><break time="0.5s"/><p>Probabilistic inference अनिश्चितता में reasoning और decision-making की प्रक्रिया है।</p><s>इसमें समस्या को define करना, model बनाना, inference करना, और परिणामों का मूल्यांकन करना शामिल है।</s><break time="0.5s"/><p>Independence का मतलब है कि variables एक दूसरे को प्रभावित नहीं करते हैं।</p><s>Marginal independence variables को अलग-अलग मानना है, जबकि absolute independence का मतलब है कि variables एक दूसरे पर condition होने पर भी independent होते हैं।</s><break time="0.5s"/><p>Bayes\' Rule probabilistic inference के लिए fundamental है।</p><s>उदाहरण के लिए, spam filtering और medical diagnosis में इसका उपयोग होता है।</s><break time="0.5s"/><p>संक्षेप में, यह मॉड्यूल AI में अनिश्चितता की अवधारणा और इससे निपटने के विभिन्न तरीकों की गहरी समझ प्रदान करता है।</p><s>आगे जानने के लिए उत्सुक रहें!</s></speak>'

In [ ]:
response = polly.synthesize_speech(
    Engine='neural',
    Text=text_input,
    TextType='ssml',
    LanguageCode='en-IN',  # 'hi-IN'
    OutputFormat='mp3',
    VoiceId='Kajal'
)

with open('speech.mp3', 'wb') as file:
    file.write(response['AudioStream'].read())

In [ ]:
# New prompt for normal speech

